In [ ]:
import os
import pandas as pd
import glob
from iocsearcher.searcher import Searcher
from tqdm import tqdm

# ==============================================================================
# CONFIGURAZIONE
# ==============================================================================
INPUT_FOLDER = "./Results_CSV"  
OUTPUT_FILE = "FINAL_IOC_REPORT.csv"

# Inizializza il cercatore di IoC
searcher = Searcher()

# ==============================================================================
# LOGICA DI ESTRAZIONE
# ==============================================================================
print(f"🔍 Inizio estrazione IoC dalla cartella: {INPUT_FOLDER}")

# Trova tutti i file CSV nella cartella
csv_files = glob.glob(os.path.join(INPUT_FOLDER, "*.csv"))
print(f"📋 Trovati {len(csv_files)} file CSV da analizzare.")

all_iocs = []
total_messages_scanned = 0

for file_path in tqdm(csv_files, desc="Analisi File"):
    try:
        # Leggiamo il CSV del gruppo (contiene solo messaggi già classificati come Cyber)
        df_group = pd.read_csv(file_path)
        
        if df_group.empty: continue
        
        # Nome del gruppo dal nome del file (per tracciabilità)
        group_name = os.path.basename(file_path).replace(".csv", "").replace("_", " ")
        
        total_messages_scanned += len(df_group)
        
        # Iteriamo su ogni messaggio
        for _, row in df_group.iterrows():
            text = str(row.get('text', '')) # Colonna 'text' contiene l'originale
            msg_date = row.get('date', '')
            user_id = row.get('user_id', '')
            
            if len(text) < 5: continue
            
            # --- ESTRAZIONE IOC (La parte che volevi recuperare) ---
            try:
                # search_raw restituisce una lista di tuple: [('URL', 'http://...'), ('IPv4', '1.2.3.4')]
                iocs_found = searcher.search_raw(text)
                
                if iocs_found:
                    for ioc_type, ioc_value in iocs_found:
                        # Salviamo l'IoC trovato
                        all_iocs.append({
                            "group_name": group_name,
                            "date": msg_date,
                            "user_id": user_id,
                            "ioc_type": ioc_type,   # Es: 'IPv4', 'URL', 'Bitcoin', 'Email'
                            "ioc_value": ioc_value, # Es: '192.168.1.1'
                            "context_text": text    # Utile per capire il contesto (es. "Attack target: [IOC]")
                        })
            except Exception as e:
                # A volte parser regex complessi possono fallire su caratteri strani
                continue
                
    except Exception as e:
        print(f"⚠️ Errore leggendo {file_path}: {e}")

# ==============================================================================
# SALVATAGGIO FINALE
# ==============================================================================
print("\n" + "="*50)
print("📊 STATISTICHE ESTRAZIONE")
print(f"- Messaggi Cyber Analizzati: {total_messages_scanned}")
print(f"- IoC Totali Estratti:       {len(all_iocs)}")

if all_iocs:
    # Creiamo DataFrame finale
    df_iocs = pd.DataFrame(all_iocs)
    
    # Rimozione duplicati (stesso IoC nello stesso messaggio = inutile)
    df_iocs = df_iocs.drop_duplicates(subset=['ioc_value', 'group_name', 'date'])
    
    print(f"- IoC Unici (dopo pulizia):  {len(df_iocs)}")
    
    # Salvataggio
    df_iocs.to_csv(OUTPUT_FILE, index=False)
    print(f"\n✅ REPORT COMPLETATO: {OUTPUT_FILE}")
    
    # Anteprima top IoC types
    print("\nTop IoC Types found:")
    print(df_iocs['ioc_type'].value_counts().head())
else:
    print("\n❌ Nessun IoC trovato nei messaggi.")

In [ ]:
import os
import glob
import pandas as pd
from tqdm import tqdm

# ==============================================================================
# CONFIGURAZIONE
# ==============================================================================
CARTELLA_INPUT = "./Results_CSV"
NOME_FILE_OUTPUT = "FULL_CYBER_DATASET_MERGED.csv"

# ==============================================================================
# LOGICA DI UNIONE
# ==============================================================================
print(f"📂 Cerco file CSV nella cartella: {CARTELLA_INPUT}...")

# Trova tutti i file .csv usando glob
files = glob.glob(os.path.join(CARTELLA_INPUT, "*.csv"))

if not files:
    print("❌ ERRORE: Nessun file CSV trovato. Controlla il percorso.")
    exit()

print(f"📋 Trovati {len(files)} file. Inizio unione...")

lista_dataframe = []
totale_messaggi = 0

# Ciclo su ogni file con barra di progresso
for file in tqdm(files, desc="Merging"):
    try:
        # Legge il singolo CSV
        df_temp = pd.read_csv(file)
        
        # Se il file non è vuoto, lo aggiungiamo alla lista
        if not df_temp.empty:
            lista_dataframe.append(df_temp)
            totale_messaggi += len(df_temp)
            
    except Exception as e:
        print(f"⚠️ Errore leggendo il file {file}: {e}")

# ==============================================================================
# CONCATENAZIONE E SALVATAGGIO
# ==============================================================================
if lista_dataframe:
    print("\n🔄 Concatenazione in corso...")
    # Unisce tutti i pezzi in un unico DataFrame gigante
    df_finale = pd.concat(lista_dataframe, ignore_index=True)
    
    # (Opzionale) Rimuove eventuali duplicati esatti se lo stesso messaggio è stato salvato due volte
    df_finale = df_finale.drop_duplicates()
    
    print(f"📊 Statistiche Finali:")
    print(f"   - File uniti: {len(files)}")
    print(f"   - Totale righe grezze: {totale_messaggi}")
    print(f"   - Totale righe uniche: {len(df_finale)}")
    print(f"   - Colonne: {list(df_finale.columns)}")
    
    # Salvataggio
    df_finale.to_csv(NOME_FILE_OUTPUT, index=False)
    print(f"\n✅ SUCCESS! Dataset completo salvato come: {NOME_FILE_OUTPUT}")
    
else:
    print("❌ Nessun dato valido trovato da unire.")

📂 Cerco file CSV nella cartella: ./Results_CSV...
📋 Trovati 12 file. Inizio unione...


Merging: 100%|██████████| 12/12 [00:02<00:00,  5.33it/s]



🔄 Concatenazione in corso...
📊 Statistiche Finali:
   - File uniti: 12
   - Totale righe grezze: 219727
   - Totale righe uniche: 219727
   - Colonne: ['message_id', 'date', 'user', 'text', 'score']

✅ SUCCESS! Dataset completo salvato come: FULL_CYBER_DATASET_MERGED.csv


In [7]:
import pandas as pd
import re
from iocsearcher.searcher import Searcher
from tqdm import tqdm

# ==============================================================================
# 1. CONFIGURAZIONE
# ==============================================================================
INPUT_FILE = "FULL_CYBER_DATASET_MERGED.csv" 
OUTPUT_FILE = "FINAL_IOC_REPORT_WHITELISTED.csv"

# --- WHITELIST (LISTA DEI BUONI) ---
# Se un URL contiene una di queste parole, viene SCARTATO.
# Puoi aggiungere o togliere domini da questa lista.
WHITELIST_DOMAINS = [
    "t.me", "telegram.me",       # Link interni Telegram
    "youtube.com", "youtu.be",   # Video
    "facebook.com", "fb.com",    # Social
    "instagram.com", 
    "twitter.com", "x.com",
    "tiktok.com",
    "linkedin.com",
    "whatsapp.com",
    "google.com", "goo.gl",      # Google Services (Attenzione: a volte usati per phishing)
    "gmail.com",
    "netflix.com",
    "spotify.com",
    "amazon.com",
    "apple.com",
    "microsoft.com",
    "github.com", "gitlab.com",  # Code repo (Attenzione: spesso ospitano malware, ma se vuoi pulire togli)
    "play.google.com",           # App Store ufficiali
    "discord.com", "discord.gg",
    "zoom.us",
    "wikipedia.org"
]

# Inizializza la libreria standard
searcher = Searcher()

# ==============================================================================
# 2. REGEX E FUNZIONI DI FILTRO
# ==============================================================================
REGEX_URL_GREEDY = r'https?://\S+' 
REGEX_IP_LOOSE = r'\b(?:\d{1,3}\.){3}\d{1,3}\b'
REGEX_MD5 = r'\b[a-fA-F0-9]{32}\b'

def is_whitelisted(ioc_value):
    """Ritorna True se l'IoC contiene un dominio in whitelist"""
    ioc_lower = str(ioc_value).lower()
    for domain in WHITELIST_DOMAINS:
        if domain in ioc_lower:
            return True
    return False

# ==============================================================================
# 3. MOTORE DI ESTRAZIONE
# ==============================================================================
print(f"🔄 Caricamento dataset: {INPUT_FILE}...")
try:
    df = pd.read_csv(INPUT_FILE)
    print(f"📋 Messaggi da analizzare: {len(df)}")
except FileNotFoundError:
    print("❌ Errore: File non trovato. Esegui prima l'unione dei CSV.")
    exit()

extracted_iocs = []
stats = {"trovati": 0, "scartati_whitelist": 0}

for _, row in tqdm(df.iterrows(), total=len(df), desc="Mining & Filtering"):
    text = str(row.get('text', ''))
    if len(text) < 5: continue
    
    meta = {
        "group_name": row.get('group_name', 'Unknown'),
        "date": row.get('date', ''),
        "user_id": row.get('user_id', ''),
        "original_text": text
    }

    # --- A. IOCSEARCHER ---
    found_standard = []
    try:
        iocs_list = searcher.search_raw(text)
        if iocs_list:
            for ioc_type, ioc_value in iocs_list:
                
                # CHECK WHITELIST
                if is_whitelisted(ioc_value):
                    stats["scartati_whitelist"] += 1
                    continue # Salta questo giro
                
                found_standard.append(ioc_value)
                extracted_iocs.append({**meta, "ioc_type": ioc_type, "ioc_value": ioc_value, "source": "lib-iocsearcher"})
                stats["trovati"] += 1
    except: pass

    # --- B. REGEX MANUALE ---
    
    # URL Sporchi
    dirty_urls = re.findall(REGEX_URL_GREEDY, text)
    for url in dirty_urls:
        # Filtro duplicati + Whitelist
        if not any(url in standard for standard in found_standard):
            if is_whitelisted(url):
                stats["scartati_whitelist"] += 1
                continue
                
            extracted_iocs.append({**meta, "ioc_type": "URL (Dirty)", "ioc_value": url, "source": "regex-greedy"})
            stats["trovati"] += 1

    # IP Loose
    loose_ips = re.findall(REGEX_IP_LOOSE, text)
    for ip in loose_ips:
        # Semplice filtro per IP locali (192.168.x.x e 127.0.0.1) che sono rumore
        if ip.startswith("192.168.") or ip.startswith("127.") or ip.startswith("10."):
            continue
            
        if ip not in found_standard:
            extracted_iocs.append({**meta, "ioc_type": "IPv4 (Loose)", "ioc_value": ip, "source": "regex-loose"})
            stats["trovati"] += 1

    # Hash
    hashes = re.findall(REGEX_MD5, text)
    for h in hashes:
        if h not in found_standard:
            extracted_iocs.append({**meta, "ioc_type": "MD5 Hash", "ioc_value": h, "source": "regex-md5"})
            stats["trovati"] += 1

# ==============================================================================
# 4. PULIZIA E SALVATAGGIO
# ==============================================================================
if extracted_iocs:
    df_result = pd.DataFrame(extracted_iocs)
    df_result = df_result.drop_duplicates(subset=['ioc_value'])
    
    print("\n" + "="*50)
    print("✅ ESTRAZIONE COMPLETATA")
    print(f"🔹 IoC Totali trovati (grezzi): {stats['trovati']}")
    print(f"🧹 IoC Scartati (Whitelist):   {stats['scartati_whitelist']}")
    print(f"🔹 IoC Unici salvati:          {len(df_result)}")
    print("="*50)
    
    print("\nTipologie rimaste:")
    print(df_result['ioc_type'].value_counts())
    
    df_result.to_csv(OUTPUT_FILE, index=False)
    print(f"\n💾 File pulito salvato: {OUTPUT_FILE}")
else:
    print("\n❌ Nessun IoC trovato dopo il filtraggio.")

🔄 Caricamento dataset: FULL_CYBER_DATASET_MERGED.csv...
📋 Messaggi da analizzare: 219727


Mining & Filtering: 100%|██████████| 219727/219727 [01:29<00:00, 2446.12it/s] 


✅ ESTRAZIONE COMPLETATA
🔹 IoC Totali trovati (grezzi): 624
🧹 IoC Scartati (Whitelist):   2546
🔹 IoC Unici salvati:          305

Tipologie rimaste:
ioc_type
URL (Dirty)     218
IPv4 (Loose)     67
MD5 Hash         20
Name: count, dtype: int64

💾 File pulito salvato: FINAL_IOC_REPORT_WHITELISTED.csv


In [6]:
df = pd.read_csv('FINAL_IOC_REPORT_HYBRID.csv')
df = df.drop_duplicates('ioc_value')
df.to_csv('FINAL_IOC_REPORT_HYBRID2.csv', index=False)